Note: this notebook should be run from the debug-shuffle branch.

In this notebook we were trying to see why the shuffle trials were getting slightly different performance when using our custom scorer.
Turns out it was because our custom scorer, in adding noise, changes the state of the np.random random number generator.
This then means that we get a differen relabeling of the trials in the shuffled setting.
This then gives us different estimators, which then means that our final performance will be slightly different.
This doesn't affect the unshuffled trials because the trials chosen for training and test are determined by StratifiedShuffleSplit.
And that uses its own RNG that's unperturbed by subsequent uses of np.random.
So all that matters for the unshuffled trials are the indices, which remain the same whether we use the custom scorer or not.

In [1]:
%pylab inline
import logging
from importlib import reload
plt.style.use("default")

Populating the interactive namespace from numpy and matplotlib


In [2]:
import run, inputs
reload(run)
reload(inputs)
Config = inputs.Config
inputs.logger.setLevel(logging.DEBUG)
run.logger.setLevel(logging.DEBUG)

In [3]:
seed = 0
search = run.get_pipeline("svc_rbf", seed=seed)
conf = Config(seed, 145, 1, 2, "CD", "yes", 0.025, 6, 1, 0.75)
run.run_single(conf, search, custom_scoring = True, scoring_noise = False)

DEBUG:run.py:Running with Config(seed=0, n_sub=145, shuf=1, freq=2, pairs='CD', whiskers='yes', window_size=0.025, start_time=6, response_threshold=1, min_resp_trials=0.75).


DEBUG:run.py:Custom scoring is TRUE.


DEBUG:run.py:Scoring noise  is FALSE.


DEBUG:inputs.py:generate_input_for_config


DEBUG:inputs.py:Generating input for Config(seed=0, n_sub=145, shuf=1, freq=2, pairs='CD', whiskers='yes', window_size=0.025, start_time=6, response_threshold=1, min_resp_trials=0.75)


...

DEBUG:run.py:Split  1: y_pred_tst=array([1, 1, 1])


DEBUG:run.py:Split  1: TRAINING (11/21 trials are +1) 0.524	TEST ( 1/3  trials are +1) 0.333


DEBUG:run.py:Train 0.524 +/- 0.000


DEBUG:run.py: Test 0.333 +/- 0.000


(0.5238095238095238, 0.3333333333333333)

Finally, just to check explictly that SSS is unperturbed by uses of np.random

In [4]:
from sklearn.model_selection import StratifiedShuffleSplit

In [5]:
X = randn(24,10);
y = 2*( randn(24,)>0)-1

In [6]:
for i, (itrn, itst) in enumerate(StratifiedShuffleSplit(random_state=0).split(X,y)):
    print(itrn)    
    a = random.randn(20,)

[23  9 14  2  0 10  1 12 13  7 20  6 18  4 11 16 21 15  5  3 17]
[21  2 14  6  7 23 19  0 15 16  8  9 17  5 10 20 11  3 22 13  1]
[17 18  6 16 15 10  7 14 20 11 21 22  9  8 13  5  1  2 23 19  0]
[ 0 13 17 15  5  4 23 22  7 10 21  8 11 16  9 19 18 20 12  1 14]
[ 8 19  1  2  3  9 18 17 21 23 12 20  5  6  7 14 15 11 10 22 16]
[12  6 10  4  1 11 18 17 20  3 23  2  5  0 21 13 16  7 19 22 15]
[15  1 18 22  8 23  4  0 11  7  6 16 13 12  9 17 14 10  3  5 21]
[13 11 19  6  4  2  1 22 21 16 23 10 14  7 15 20 12  3  0 18  9]
[ 7 16  0 22 19  4 20 17 13 11  5 10  2  3  8 18  1 21  9 12  6]
[11  8 17 21  0  7 19  1 12  6 13  2 23  9 18 22 20  4  5 16  3]
